In [1]:
import pandas as pd
import requests
from datetime import datetime
import time
import multiprocessing
import json
import psycopg2

import os

from dotenv import load_dotenv
load_dotenv()

from bs4 import BeautifulSoup
pd.set_option('display.max_rows', None)

## Helper Functions

In [3]:
def coleta_html(link):
    headers = {'user-agent': 'Mozilla/5.0'}
    resposta = requests.get(link,
                            headers= headers
                            )
    proposta_soup = BeautifulSoup(resposta.text)

    return proposta_soup


def coleta_html_detalhado(link, nome_vereador):
    #print('1')
    if verifica_link(link, nome_vereador):
        return 0
    else:
        headers = {'user-agent': 'Mozilla/5.0'}
        resposta = requests.get(link,
                                headers= headers
                                )
        proposta_soup = BeautifulSoup(resposta.text)

        return proposta_soup
        

def conecta_banco_prefeitura():
    dbname = os.getenv("DBNAME")
    user = os.getenv("USER")
    password = os.getenv("PASSWORD")
    host = os.getenv("HOST")
    port = os.getenv("PORT")

    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    #cur = conn.cursor()

    return conn




def verifica_link(link, nome_vereador):

    conn = conecta_banco_prefeitura()
    cur = conn.cursor()

    cur.execute("SELECT link FROM links_proprosituras_vereadores WHERE nome_vereador =  %s", (nome_vereador,))

    results = cur.fetchall()

    df_teste = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])

    conn.commit()
    cur.close()
    conn.close()

    if link in list(df_teste['link']):
        return True 
    else:
        return False

def insere_link(link, nome):
    conn = conecta_banco_prefeitura()
    cur = conn.cursor()

    cur.execute("INSERT INTO links_proprosituras_vereadores (nome_vereador, link) VALUES (%s, %s);", (nome, link))
    conn.commit()
    cur.close()
    conn.close()

def insert_proposituras_detalhadas(nome, tipo, situacao_content, data_content):
    conn = conecta_banco_prefeitura()
    cur = conn.cursor()

    cur.execute("INSERT INTO proposituras_detalhadas (nome_vereador, tipo, situacao, data_propositura) VALUES (%s, %s, %s, %s);", (nome, tipo, situacao_content, data_content))
    conn.commit()
    cur.close()
    conn.close()

In [4]:
headers = {'user-agent': 'Mozilla/5.0'}
resposta = requests.get('https://www.camarabotucatu.sp.gov.br/Vereador',
                        headers= headers
                        )

## Pegando o Nome e o Partido dos Vereadores

In [5]:
text_estruturado = BeautifulSoup(resposta.text)
#text_estruturado

In [6]:
lista_vereadores = text_estruturado.find_all('h2')

In [7]:
lista_vereadores

[<h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/68" target="_blank">Abelardo Wanderlino da Costa Neto - Abelardo</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/81" target="_blank">Alessandra Lucchesi de Oliveira - Alessandra Lucchesi</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/69" target="_blank">André Rogério Barbosa - Curumim (Licenciado)</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/47" target="_blank">Antonio Carlos Vaz de Almeida - Cula</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/7076" target="_blank">Cláudia Maria Gabriel - Cláudia Gabriel (Licenciada)</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/7075" target="_blank">Elias Marcelo Sleiman - Marcelo Sleiman</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp

In [8]:
# Captando o nome e o partido dos vereadores
tags_li = text_estruturado.find_all('span')
lista_partidos = []
for i in range(len(tags_li)):
    if tags_li[i].find_all(['li']):
        if tags_li[i].find_all('a'):
            lista_partidos.append( tags_li[i].text.replace('\n', '') )


nomes_vereadores = []
for i in range(len(lista_vereadores)):
    #print(i)
    nomes_vereadores.append(  lista_vereadores[i].contents[1].text.split('-')[1].strip()  )


## Captando as imagens dos Vereadores

In [9]:
# import re

# teste = []
# teste2 = []

# x = text_estruturado.find_all('div', {'class': 'col-md-3'})
# for i in range(len(x)):
#     if x[i].find('div', {'class':'img'}):
#         teste.append(x[i].find('div', {'class':'img'}))
#         #print(teste)
# for j in range(len(teste)):
#     if teste[j].find('img'):
#         teste2.append(teste[j].find_all('img', {'src': re.compile('\.jpg$', re.IGNORECASE)}))


In [10]:
# save_folder = "../img"
# for img_tag in teste2:
#     src = img_tag[0]['src']
#     alt = img_tag[0]['alt'].split('-')[1].strip()
#     img_url = f"https://www.camarabotucatu.sp.gov.br/{src}" 
#     filename = os.path.join(save_folder, f"{alt}.jpg")

#     response = requests.get(img_url)
#     with open(filename, 'wb') as f:
#         f.write(response.content)

In [11]:
df_vereadores = pd.DataFrame()
lista_partidos

df_vereadores['nome_vereador'] = nomes_vereadores
df_vereadores['partido_vereador'] = lista_partidos

In [12]:
df_vereadores

,nome_vereador,partido_vereador
0,Abelardo,Republicanos
1,Alessandra Lucchesi,PSDB-Partido da Social Democracia Brasileira
2,Curumim (Licenciado),PSDB-Partido da Social Democracia Brasileira
3,Cula,PSDB-Partido da Social Democracia Brasileira
4,Cláudia Gabriel (Licenciada),União Brasil
5,Marcelo Sleiman,União Brasil
6,Erika da Liga do Bem,Republicanos
7,Pedroso (Suplente),União Brasil
8,Sargento Laudo,PSDB-Partido da Social Democracia Brasileira
9,Lelo Pagani (Suplente),PSDB-Partido da Social Democracia Brasileira


## Para Cada Vereador trazer tabela do que ja fez em cada Ano

In [13]:
# links_vereadores = []
# for i in range(len(lista_vereadores)):
#     links_vereadores.append(lista_vereadores[i].a.get('href'))

# df_vereadores['link'] = links_vereadores

In [14]:
# df_final = pd.DataFrame(columns=['Tipo', 'Quantidade','Ano'])

# for i in range(len(df_vereadores)):

#     print(i)

#     #pega o nome do vereador
#     nome_atual_vereador = df_vereadores.iloc[i][:i+1]['nome_vereador']


#     #pega o link do vereador
#     #df_vereadores.iloc[i][:i+1]['link']


#     headers = {'user-agent': 'Mozilla/5.0'}
#     resposta_vereador = requests.get(df_vereadores[:i+1]['link'].iloc[-1],
#                         headers= headers
#                         )
    
#     vereador_estruturado = BeautifulSoup(resposta_vereador.text)
#     tabela_documentos = vereador_estruturado.find_all('table')

#     # Pegando a linha da tabela do site que contém os anos de atuação dos vereadores
#     lista_anos = []

#     for i in range(len(tabela_documentos)):
#         if tabela_documentos[i].find_all('caption')[0].text == 'Documentos':
#             tabela_documentos = tabela_documentos[i]


#     for i in range(len( tabela_documentos.find_all('th', {'class': 'text-right'}) ) ):
#         if tabela_documentos.find_all('th', {'class': 'text-right'})[i].text:
#             lista_anos.append(tabela_documentos.find_all('th', {'class': 'text-right'})[i].text)
#     lista_anos = sorted([*set(lista_anos)])
#     if 'Total' in lista_anos:
#         lista_anos.remove('Total')
#     #lista_anos


#     # Pegando todas as proposituras do vereador em cada tipo e alocando em um dataframe
#     tbody = tabela_documentos.find_all('tbody')[0].find_all('tr')#[0]#.find_all('td')
#     list_dict = []
#     df_teste = pd.DataFrame(columns=['Tipo', 'Quantidade'])

#     for i in range(len(tbody)):
#         for j in range(len( tbody[i].find_all('td') )):
#             list_dict.append(tbody[i].find_all('td')[j].text)
#         #print(list_dict)
        
#         #list_aux = list_dict
#         #list_aux.remove(list_aux[0])
#         df_novo = pd.DataFrame({'Tipo': [list_dict[0]]*(len(list_dict)-2), 'Quantidade': list_dict[1:-1], 'Ano': lista_anos,'Vereador': nome_atual_vereador})
#         df_teste = pd.concat([df_teste, df_novo], ignore_index=True)

#         list_dict = []
#     df_final = pd.concat([df_final,df_teste])
#     df_final = df_final.loc[df_final['Tipo'] != 'Total']
    
    

In [15]:
# df_final['Quantidade'] = df_final['Quantidade'].apply(lambda x: 0 if x == '-' else x)
# df_final.head()

In [16]:
# df_final['Quantidade'] = df_final['Quantidade'].astype('int')
# df_final['Ano'] = df_final['Ano'].astype('str')
# df_final.dtypes

## Coletando detalhadamente dados dos projetos apresentados

In [17]:
headers = {'user-agent': 'Mozilla/5.0'}
detalhamento_propostas = requests.get('https://www.camarabotucatu.sp.gov.br/Consulta/Vereadores/',
                        headers= headers
                        )



In [18]:
texto_detalhamento = BeautifulSoup(detalhamento_propostas.text)
itens_vereadores = texto_detalhamento.find_all('div', {'class': 'data-list-item data-list-striped data-list-hover'})

In [19]:
# Pegando o link que leva a listagem de proposituras
lista_link_propositura = []
for i in range(len(itens_vereadores)):
    x = itens_vereadores[i].find_all('a')
    for j in range(len(x)):
        if x[j].text == 'Detalhadas':
            lista_link_propositura.append(x[j].get('href'))
    

In [20]:
lista_link_propositura

['/Consulta/Vereadores/Proposituras/68',
 '/Consulta/Vereadores/Proposituras/81',
 '/Consulta/Vereadores/Proposituras/47',
 '/Consulta/Vereadores/Proposituras/7075',
 '/Consulta/Vereadores/Proposituras/7072',
 '/Consulta/Vereadores/Proposituras/7338',
 '/Consulta/Vereadores/Proposituras/83',
 '/Consulta/Vereadores/Proposituras/67',
 '/Consulta/Vereadores/Proposituras/7073',
 '/Consulta/Vereadores/Proposituras/80',
 '/Consulta/Vereadores/Proposituras/7074']

In [21]:
df_link_proposituras = pd.DataFrame()


# Pegar apenas a proposta de vereadores ativos no momento e não pegar Licenciados
lista_vereadores = list(df_vereadores['nome_vereador'])
lista_final_vereadores = []
for i in range(len(lista_vereadores)):
    if 'Licenciad' not in lista_vereadores[i]:
        lista_final_vereadores.append(lista_vereadores[i])

df_link_proposituras['vereador'] = lista_final_vereadores
df_link_proposituras['link_proposituras'] = lista_link_propositura
#df_link_proposituras



In [22]:
lista_link_propositura

['/Consulta/Vereadores/Proposituras/68',
 '/Consulta/Vereadores/Proposituras/81',
 '/Consulta/Vereadores/Proposituras/47',
 '/Consulta/Vereadores/Proposituras/7075',
 '/Consulta/Vereadores/Proposituras/7072',
 '/Consulta/Vereadores/Proposituras/7338',
 '/Consulta/Vereadores/Proposituras/83',
 '/Consulta/Vereadores/Proposituras/67',
 '/Consulta/Vereadores/Proposituras/7073',
 '/Consulta/Vereadores/Proposituras/80',
 '/Consulta/Vereadores/Proposituras/7074']

In [29]:
def coleta_propostituras_vereadores(nome_vereador, link_proposituras_detalhadas):
    #print('FUNÇÃO!!')
    resultado = []
    dict_data = {}
    z = 0

    
    nome = nome_vereador
    link = link_proposituras_detalhadas

    propostas_detalhadas = coleta_html('https://www.camarabotucatu.sp.gov.br'+ link)

    a = propostas_detalhadas.find_all('div', {'class': 'data-list-item data-list-striped data-list-hover'})

    for i in range(len(a)):
        tipo = a[i].h3.text
        z = 1
        
        for j in range(len(a[i].find_all('p'))):
            #print('Vereador: '+ nome + ' ' + tipo.text + ': '+str(z) )
            z= z + 1
            #html_soup = coleta_html('https://www.camarabotucatu.sp.gov.br'+a[i].find_all('p')[j].a.get('href'))

            link = 'https://www.camarabotucatu.sp.gov.br'+a[i].find_all('p')[j].a.get('href')
            html_soup = coleta_html_detalhado(link, nome)

            if html_soup != 0:
                insere_link(link, nome)

                situacao = html_soup.find('strong', text='Situação:')
            
                if situacao:
                    data_strong = html_soup.find('strong', text='Data:')
                    
                    situacao_content = situacao.find_next_sibling(text=True).strip()
                    data_content = data_strong.find_next_sibling(text=True).strip()

                    data_content = datetime.strptime(data_content, '%d/%m/%Y')
                    

                    if data_content.year >= 2021:

                        dict_data = {
                            'Vereador': nome,
                            'Tipo': tipo,
                            'Situação': situacao_content,
                            'Data': data_content
                        }

                        insert_proposituras_detalhadas(nome, tipo, situacao_content, data_content)

                        resultado.append(dict_data)
                    else: 
                        continue_outer_loop = True
                        break

                else:
                    continue_outer_loop = True
            else:
                #print("Já Existe:")
                continue_outer_loop = True
                break

        time.sleep(10)
    #return  resultado

    

In [40]:
df_link_proposituras.loc[df_link_proposituras['vereador'] == 'Abelardo']

,vereador,link_proposituras
0,Abelardo,/Consulta/Vereadores/Proposituras/68


In [38]:
# # coleta_propostituras_vereadores(nome_vereador, link_proposituras_detalhadas)

for index, row in df_link_proposituras.loc[df_link_proposituras['vereador'] == 'Abelardo'].iterrows():
    resultado = []
    dict_data = {}
    z = 0

    
    nome = row['vereador']
    link = row['link_proposituras']

    propostas_detalhadas = coleta_html('https://www.camarabotucatu.sp.gov.br'+ link)

    a = propostas_detalhadas.find_all('div', {'class': 'data-list-item data-list-striped data-list-hover'})

    for i in range(len(a)):
        tipo = a[i].h3.text
        z = 1
        
        for j in range(len(a[i].find_all('p'))):
            #print('Vereador: '+ nome + ' ' + tipo.text + ': '+str(z) )
            z= z + 1

            link = 'https://www.camarabotucatu.sp.gov.br'+a[i].find_all('p')[j].a.get('href')
            #resultado.append(link)


In [39]:
resultado

[]

In [36]:
df_link_proposituras

,vereador,link_proposituras
0,Abelardo,/Consulta/Vereadores/Proposituras/68
1,Alessandra Lucchesi,/Consulta/Vereadores/Proposituras/81
2,Cula,/Consulta/Vereadores/Proposituras/47
3,Marcelo Sleiman,/Consulta/Vereadores/Proposituras/7075
4,Erika da Liga do Bem,/Consulta/Vereadores/Proposituras/7072
5,Pedroso (Suplente),/Consulta/Vereadores/Proposituras/7338
6,Sargento Laudo,/Consulta/Vereadores/Proposituras/83
7,Lelo Pagani (Suplente),/Consulta/Vereadores/Proposituras/67
8,PALHINHA,/Consulta/Vereadores/Proposituras/7073
9,Rose Ielo,/Consulta/Vereadores/Proposituras/80


In [35]:
len(resultado)

752

In [25]:
# import concurrent.futures

# if __name__ == '__main__':
#     # Lista de vereadores
#     vereadores = df_link_proposituras.to_dict(orient='records')

#     # Criação do executor
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Criação das tarefas
#         try:
#             tarefas = [executor.submit(coleta_propostituras_vereadores, vereador['vereador'], vereador['link_proposituras']) for vereador in vereadores]

#             # Coleta dos resultados
#             resultados = [tarefa.result() for tarefa in concurrent.futures.as_completed(tarefas)]

#         # Imprime os resultados
#             print(resultados)
#         except psycopg2.InterfaceError as e:
#             time.sleep(60)
#             print("Iniciando o Processo Novamente:", e)

#             tarefas = [executor.submit(coleta_propostituras_vereadores, vereador['vereador'], vereador['link_proposituras']) for vereador in vereadores]
#             # Coleta dos resultados
#             resultados = [tarefa.result() for tarefa in concurrent.futures.as_completed(tarefas)]
#         except psycopg2.OperationalError as e:
#             time.sleep(60)
#             print("Iniciando o Processo Novamente:", e)

#             tarefas = [executor.submit(coleta_propostituras_vereadores, vereador['vereador'], vereador['link_proposituras']) for vereador in vereadores]
#             # Coleta dos resultados
#             resultados = [tarefa.result() for tarefa in concurrent.futures.as_completed(tarefas)]


In [1]:
resultados

NameError: name 'resultados' is not defined

In [69]:



# import multiprocessing


# if __name__ == '__main__':
#     # Lista de dicionários
#     vereadores = df_link_proposituras.to_dict(orient='records')

#     # Criação da lista compartilhada
#     manager = multiprocessing.Manager()
#     resultado_compartilhado = manager.list()

#     # Criação da lista de tuplas de parâmetros
#     parametros = [(vereador['vereador'], vereador['link_proposituras']) for vereador in vereadores]

#     # Criação dos processos
#     processos = []
#     for parametro in parametros:
#         processo = multiprocessing.Process(target=coleta_propostituras_vereadores, args=parametro+(resultado_compartilhado,))
#         processos.append(processo)

#     # Início dos processos
#     for processo in processos:
#         processo.start()

#     # Espera a finalização dos processos
#     for processo in processos:
#         processo.join()

#     # Juntar todas as listas em uma única
#     resultado_final = list(resultado_compartilhado)

#     # Imprimir o resultado final
#     print(resultado_final)







[]


In [26]:
# lista_dict = []
# dict_data = {}
# z = 0

# for indice, linha in df_link_proposituras.iterrows():
#     nome = linha['vereador']
#     link = linha['link_proposituras']

#     propostas_detalhadas = coleta_html('https://www.camarabotucatu.sp.gov.br'+ link)

#     a = propostas_detalhadas.find_all('div', {'class': 'data-list-item data-list-striped data-list-hover'})

#     for i in range(len(a)):
#         tipo = a[i].h3
#         z = 1
        
#         for j in range(len(a[i].find_all('p'))):
#             #print(a[i].find_all('p')[j].a.get('href') )
#             print('Vereador: '+ nome + ' ' + tipo.text + ': '+str(z) )
#             z= z + 1
#             html_soup = coleta_html('https://www.camarabotucatu.sp.gov.br'+a[i].find_all('p')[j].a.get('href'))

#             situacao = html_soup.find('strong', text='Situação:')
            
#             if situacao:
#                 data_strong = html_soup.find('strong', text='Data:')
                
#                 situacao_content = situacao.find_next_sibling(text=True).strip()
#                 data_content = data_strong.find_next_sibling(text=True).strip()

#                 data_content = datetime.strptime(data_content, '%d/%m/%Y')

#                 if data_content.year >= 2021:
                
#                     dict_data = {
#                         'Vereador': nome,
#                         'Tipo': tipo,
#                         'Situação': situacao_content,
#                         'Data': data_content
#                     }

#                     lista_dict.append(dict_data)

#                     time.sleep(50)

#                     with open('dados.json', 'r+') as arquivo_json:
#                         dados = json.load(arquivo_json)

#                         dados.append(dict_data)

#                         arquivo_json.seek(0)
                
#                 else: 
#                     continue_outer_loop = True
#                     break

#             else:
#                 print('')
#     time.sleep(60)
        


Vereador: Abelardo Correspondências Recebidas (2): 1

Vereador: Abelardo Correspondências Recebidas (2): 2

Vereador: Abelardo Emendas (35): 1
Vereador: Abelardo Emendas (35): 2
Vereador: Abelardo Emendas (35): 3
Vereador: Abelardo Emendas (35): 4
Vereador: Abelardo Emendas (35): 5
Vereador: Abelardo Emendas (35): 6
Vereador: Abelardo Emendas (35): 7
Vereador: Abelardo Emendas (35): 8
Vereador: Abelardo Emendas (35): 9
Vereador: Abelardo Indicações (71): 1
Vereador: Abelardo Indicações (71): 2
Vereador: Abelardo Indicações (71): 3
Vereador: Abelardo Indicações (71): 4
Vereador: Abelardo Indicações (71): 5
Vereador: Abelardo Indicações (71): 6
Vereador: Abelardo Indicações (71): 7
Vereador: Abelardo Indicações (71): 8
Vereador: Abelardo Indicações (71): 9
Vereador: Abelardo Indicações (71): 10
Vereador: Abelardo Indicações (71): 11
Vereador: Abelardo Indicações (71): 12
Vereador: Abelardo Indicações (71): 13
Vereador: Abelardo Indicações (71): 14
Vereador: Abelardo Indicações (71): 15
V

## Salvando no Banco de Dados

In [23]:
# import psycopg2

dbname = os.getenv("DBNAME")
user = os.getenv("USER")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")
port = os.getenv("PORT")

# conn = psycopg2.connect(
#     host=host,
#     port=port,
#     user=user,
#     password=password
# )

# #conn.set_isolation_level(0)
# #cur = conn.cursor()
# #cur.execute('CREATE DATABASE prefeitura')
# #conn.commit()
# #cur.close()
# #conn.close()



In [24]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

In [25]:
# # #cur = conn.cursor()

# cur.execute("DELETE FROM links_proprosituras_vereadores")

# # #insert_proposituras_detalhadas(nome, tipo, situacao_content, data_content)
# # #cur.execute("CREATE TABLE proposituras_detalhadas (nome_vereador VARCHAR(50), tipo VARCHAR(50), situacao VARCHAR(50), data_propositura DATE  ) ")

# # #cur.execute("CREATE TABLE vereadores (nome_vereador VARCHAR(50), partido_vereador VARCHAR(50), link_vereador VARCHAR(200) );")
# # #cur.execute("CREATE TABLE proposituras (Tipo VARCHAR(50), Quantidade INTEGER, Ano VARCHAR(50), Vereador VARCHAR(50));")

# # #cur.execute("INSERT INTO tabela1 (coluna1, coluna2) VALUES (%s, %s);", (10, "texto"))
# # #cur.execute("INSERT INTO tabela2 (coluna1, coluna2) VALUES (%s, %s);", ("texto", 3.14))

# conn.commit()

# cur.close()
# conn.close()

In [23]:
# from sqlalchemy import create_engine

# engine = create_engine('postgresql://postgres:'+password+'@'+host+':'+port+'/'+dbname)
# #df_vereadores.to_sql('vereadores', engine, if_exists='replace', index=False)
# df_final.to_sql('proposituras', engine, if_exists='replace', index=False)

470

In [34]:
# ## TABELA DE IMAGENS DOS VEREADORES!!!
# cur = conn.cursor()

# cur.execute('''
#     CREATE TABLE imagens_vereadores (
#         vereador VARCHAR(50),
#         imagem BYTEA
#     );
# ''')

In [29]:
# import os
# import psycopg2
# from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [35]:
# diretorio = "C:/Users/thale/Documents/Estudos/Projeto Prefeitura"
# for arquivo in os.listdir(diretorio):
#     if arquivo.endswith(".jpg") or arquivo.endswith(".png"):
#         with open(os.path.join(diretorio, arquivo), "rb") as imagem_arquivo:
#             conteudo_imagem = imagem_arquivo.read()
#             arquivo1 = arquivo.split('.')[0]
#             cur.execute("INSERT INTO imagens_vereadores (vereador, imagem) VALUES (%s, %s)", (arquivo1, psycopg2.Binary(conteudo_imagem)))
#             conn.commit()
#         print(arquivo)

Abelardo.jpg
Alessandra Lucchesi.jpg
Cláudia Gabriel (Licenciada).jpg
Cula.jpg
Curumim (Licenciado).jpg
Erika da Liga do Bem.jpg
Lelo Pagani (Suplente).jpg
Marcelo Sleiman.jpg
PALHINHA.jpg
Pedroso (Suplente).jpg
Rose Ielo.jpg
Sargento Laudo.jpg
Silvio.jpg


In [36]:
# cur = conn.cursor()
# cur.execute('SELECT * FROM imagens_vereadores')

# results = cur.fetchall()
# for row in results:
#     print(row)

('Abelardo', <memory at 0x000002547862C580>)
('Alessandra Lucchesi', <memory at 0x000002547862C640>)
('Cláudia Gabriel (Licenciada)', <memory at 0x000002547862C700>)
('Cula', <memory at 0x000002547862C7C0>)
('Curumim (Licenciado)', <memory at 0x000002547862C880>)
('Erika da Liga do Bem', <memory at 0x000002547862C940>)
('Lelo Pagani (Suplente)', <memory at 0x000002547862CA00>)
('Marcelo Sleiman', <memory at 0x000002547862CAC0>)
('PALHINHA', <memory at 0x000002547862CB80>)
('Pedroso (Suplente)', <memory at 0x000002547862CC40>)
('Rose Ielo', <memory at 0x000002547862CD00>)
('Sargento Laudo', <memory at 0x000002547862CDC0>)
('Silvio', <memory at 0x000002547862CE80>)
